In [1]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
from nwbwidgets.allen import AllenRasterWidget
from nwbwidgets.allen import AllenPSTHWidget
import pandas as pd
import numpy as np
import sys
import os

# Add parent directory to the system path
sys.path.append(os.path.abspath(".."))
from helpers import *
from generate import *
from pathlib import Path
from plot import *

io = NWBHDF5IO('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Mice_data/context/AB122_20240804_134554.nwb')
nwbfile = io.read()


""" 
# Load the file with the mice data
nwbfiles = []

paths = ['/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', 
        '/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB087_20231017_141901.nwb']
mouse_names = ['AB077_20230531_143839', 'AB087_20231017_141901']

for path in paths:
    io = NWBHDF5IO(path)
    nwbfiles.append(io.read())

io = NWBHDF5IO('/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839', mode='r')
nwbfile = io.read()
# To do : create a list of mouse names, to be used for the .py file.
mouse_name = 'AB077_20230531_143839'
"""

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)  # Show all columns


/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


In [16]:
import pandas as pd
df = pd.read_csv('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Data/AB129_20240828_112850/AB129_20240828_112850_AUC_Selectivity_pre_post.csv')
df2 = pd.read_csv('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Data/AB117_20240723_125437/AB117_20240723_125437_AUC_Selectivity_pre_post.csv')

In [ ]:
pd.read_parquet('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Data/AB117_20240723_125437/AB117_20240723_125437_Selectivity_Dataframe.parquet')

In [18]:
df2[df2['event']=='whisker']

,cluster_id,ccf_acronym,ccf_name,ccf_parent_id,ccf_parent_acronym,ccf_parent_name,spike_times,mouse_id,context,event,pre_spikes,post_spikes,AUC,Transformed AUC,p-values positive,p-values negative,selective,direction,pre_time,post_time
0,7,CP,Caudoputamen,485.0,STRd,Striatum dorsal region,[ 38.84138385 48.33584212 48.72360164 ......,AB117_20240723_125437,active,whisker,[0 0 6 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0],[ 0 0 10 1 1 1 0 0 0 0 0 0 0 0 0 ...,0.479873,-0.040254,0.815,0.188,False,none,0.2,0.2
1,9,CP,Caudoputamen,485.0,STRd,Striatum dorsal region,[ 13.48491803 18.81165315 19.24004525 ......,AB117_20240723_125437,active,whisker,[0 0 1 2 0 0 0 0 4 0 0 0 6 2 0 0 3 0 8 3],[0 0 3 6 1 1 0 2 1 0 1 1 4 0 2 0 3 5 6 1],0.519301,0.038601,0.903,0.099,False,none,0.2,0.2
2,13,CP,Caudoputamen,485.0,STRd,Striatum dorsal region,[1.55697129e-01 4.52591655e-01 6.55354583e-01 ...,AB117_20240723_125437,active,whisker,[1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1],[0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0],0.462621,-0.074757,0.029,0.975,True,positive,0.2,0.2
3,15,CP,Caudoputamen,485.0,STRd,Striatum dorsal region,[2.39825578e+00 9.96304964e+00 1.08213338e+01 ...,AB117_20240723_125437,active,whisker,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],0.492134,-0.015731,0.495,1.000,False,none,0.2,0.2
4,18,CP,Caudoputamen,485.0,STRd,Striatum dorsal region,[ 8.86066996 8.87836963 13.47955146 ......,AB117_20240723_125437,active,whisker,[0 0 0 0 0 1 1 0 0 0 0 1 2 2 0 0 0 0 8 0],[0 0 1 4 1 0 2 1 1 0 0 1 0 2 0 0 1 1 7 0],0.476370,-0.047260,0.905,0.098,False,none,0.2,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,434,ACAd5,"Anterior cingulate area, dorsal part, layer 5",39.0,ACAd,"Anterior cingulate area, dorsal part",[ 20.12526596 22.89774746 28.64507335 ......,AB117_20240723_125437,passive_post,whisker,[0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 1 2],[0 1 0 0 0 0 0 0 0 7 1 2 0 1 0 0 0 3 0 0 1 1],0.511832,0.023663,0.939,0.071,False,none,0.2,0.2
1523,435,MOs5,"Secondary motor area, layer 5",993.0,MOs,Secondary motor area,[1.15364079e-01 2.16496020e-01 2.61328736e-01 ...,AB117_20240723_125437,passive_post,whisker,[0 2 2 2 1 2 0 1 4 0 0 2 0 0 1 1 1 0 0 1 1 2],[1 1 2 2 2 2 0 2 1 2 0 1 2 0 0 0 1 0 2 1 0 2],0.470091,-0.059819,0.674,0.361,False,none,0.2,0.2
1524,437,MOs5,"Secondary motor area, layer 5",993.0,MOs,Secondary motor area,[1.74429932e-01 3.92493597e-01 4.46726184e-01 ...,AB117_20240723_125437,passive_post,whisker,[0 1 2 1 0 0 1 0 0 1 0 0 2 1 2 0 0 0 0 0 0 0],[0 0 0 0 0 1 0 0 2 1 0 0 1 0 1 0 1 0 0 0 0 0],0.498116,-0.003768,0.245,0.841,False,none,0.2,0.2
1525,439,MOs5,"Secondary motor area, layer 5",993.0,MOs,Secondary motor area,[ 17.78777514 31.14852988 35.40054735 5...,AB117_20240723_125437,passive_post,whisker,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],0.495935,-0.008130,1.000,1.000,False,none,0.2,0.2


In [10]:

# Check if the entire row is duplicated
has_duplicates = df2.duplicated().any()

In [11]:
has_duplicates

False

In [7]:
df[df['event']=='spontaneous_licks']['context'].unique()

array(['active', 'passive_pre', 'passive_post'], dtype=object)

In [4]:
pd.read_parquet('/Volumes/LaCie/EPFL/Mastersem3/Semester Project Lsens/Data/AB129_20240828_112850/AB129_20240828_112850_Selectivity_Dataframe.parquet')

,cluster_id,firing_rate,ccf_acronym,ccf_name,ccf_parent_acronym,ccf_parent_id,ccf_parent_name,spike_times,whisker_active_pre_spikes,whisker_active_post_spikes,...,auditory_active_post_spikes,auditory_passive_pre_pre_spikes,auditory_passive_pre_post_spikes,auditory_passive_post_pre_spikes,auditory_passive_post_post_spikes,spontaneous_licks_pre_spikes,spontaneous_licks_post_spikes,pre_time,post_time,mouse_id
0,37,6.75734934,AUDd5,"Dorsal auditory area, layer 5",AUDd,1011.0,Dorsal auditory area,"[0.054865610222700756, 0.24989518823790247, 0....","[3, 2, 0, 3, 1, 1, 3, 2, 4, 2, 4, 3, 3, 2, 1, ...","[4, 2, 1, 1, 4, 1, 2, 3, 2, 1, 2, 2, 2, 3, 8, ...",...,"[4, 2, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 3, 4, 2, ...","[0, 1, 0, 5, 0, 1, 1, 3, 1, 1, 1, 5, 3, 0, 4, ...","[2, 2, 0, 0, 2, 2, 1, 4, 0, 2, 2, 0, 0, 3, 1, ...","[1, 1, 0, 0, 4, 1, 0, 1, 3, 1, 1, 1]","[0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]","[4, 1, 2, 2, 3, 7, 4, 2, 0, 2, 3, 3, 4, 0, 4, ...","[1, 5, 6, 4, 2, 3, 1, 5, 4, 5, 5, 0, 2, 0, 2, ...",0.2,0.2,AB129_20240828_112850
1,38,2.26559196,AUDd5,"Dorsal auditory area, layer 5",AUDd,1011.0,Dorsal auditory area,"[16.011991687161927, 16.096923385125137, 16.91...","[1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1, 2, 2, 0, 0, ...","[1, 0, 1, 1, 1, 1, 4, 2, 2, 0, 0, 3, 0, 1, 0, ...",...,"[0, 0, 0, 0, 2, 0, 1, 2, 1, 1, 0, 0, 2, 1, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 1, 1, 0, 0, ...",0.2,0.2,AB129_20240828_112850
2,42,5.92652476,AUDd5,"Dorsal auditory area, layer 5",AUDd,1011.0,Dorsal auditory area,"[0.15009710986198144, 0.5737556189327752, 0.60...","[1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 1, ...","[1, 3, 0, 2, 1, 0, 4, 0, 0, 1, 2, 0, 0, 0, 1, ...",...,"[0, 1, 1, 3, 2, 0, 1, 1, 0, 0, 1, 2, 0, 4, 1, ...","[2, 0, 12, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0, 2, 0,...","[4, 0, 7, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 0, 0, ...","[1, 0, 5, 0, 1, 1, 0, 2, 2, 6, 3, 1]","[1, 2, 2, 3, 1, 1, 0, 0, 1, 4, 2, 3]","[1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 3, 0, 0, 2, ...","[0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 2, 1, 2, 2, ...",0.2,0.2,AB129_20240828_112850
3,43,5.12921274,AUDd5,"Dorsal auditory area, layer 5",AUDd,1011.0,Dorsal auditory area,"[0.23382883093089046, 0.699286535153122, 0.896...","[0, 1, 1, 0, 0, 0, 4, 0, 0, 0, 1, 2, 2, 0, 0, ...","[0, 1, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 2, 0, 0, ...",...,"[0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 4, 1, 0, 0, ...","[1, 3, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[4, 8, 3, 5, 17, 1, 4, 0, 0, 0, 2, 3, 3, 1, 3,...","[3, 3, 5, 0, 3, 2, 1, 2, 3, 2, 2, 4]","[1, 2, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1]","[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, ...","[1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",0.2,0.2,AB129_20240828_112850
4,44,3.48225893,AUDd5,"Dorsal auditory area, layer 5",AUDd,1011.0,Dorsal auditory area,"[0.23859540581658076, 0.4980570765173721, 1.80...","[0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",...,"[9, 7, 7, 3, 6, 6, 4, 4, 8, 5, 2, 1, 2, 0, 1, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[7, 4, 10, 10, 5, 9, 5, 3, 5, 7, 3, 6, 4, 6, 7...","[5, 1, 2, 2, 0, 1, 1, 1, 2, 1, 0, 1]","[1, 0, 1, 0, 0, 1, 0, 3, 3, 1, 2, 1]","[2, 2, 0, 3, 1, 0, 2, 0, 2, 0, 0, 3, 1, 4, 1, ...","[3, 3, 4, 2, 1, 1, 2, 1, 5, 1, 0, 3, 2, 5, 1, ...",0.2,0.2,AB129_20240828_112850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,617,0.50985671,SCop,"Superior colliculus, optic layer",SCs,302.0,"Superior colliculus, sensory related","[1.8759966955282446, 16.176587828996034, 16.20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0

### Loading the tables and preprocessing


#### For each line of trials table, we start by testing 10 neurons. In other words, if there is a stimulus (whisker or auditory) at a certain start time then, compute the number of spikes for each trial for each neuron.

In [15]:
units = nwbfile.units.to_dataframe()
trials = nwbfile.trials.to_dataframe()

# Preprocessing s
filtered_units = units[(units['bc_label'] == 'good') & (units['ccf_acronym'].str.contains('[A-Z]'))]


# Some chosen columns
#cons_columns  = ["cluster_id", "firing_rate", "ccf_acronym", "ccf_name", "ccf_parent_acronym", "ccf_parent_name", "spike_times"]#, "lick_flag", "lick_time"]
#filtered_units = filtered_units[cons_columns]

In [112]:
trials[trials['context']=='passive']

,start_time,stop_time,trial_type,perf,whisker_stim,whisker_stim_amplitude,whisker_stim_duration,whisker_stim_time,auditory_stim,auditory_stim_amplitude,auditory_stim_frequency,auditory_stim_duration,auditory_stim_time,no_stim,no_stim_time,stim_onset,reward_available,response_window_start_time,response_window_stop_time,lick_flag,lick_time,abort_window_start_time,abort_window_stop_time,early_lick,context,context_background,opto_stim,opto_grid_ap,opto_grid_ml,opto_grid_no,opto_stim_start_time,opto_stim_stop_time,opto_stim_amplitude,opto_stim_frequency,trial_id
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,22.705635,23.805635,auditory_trial,6,0,0.0,0,NaN,1,10,10000,0,22.705635,0,NaN,22.705635,1,22.805635,23.805635,0,NaN,22.705635,22.705635,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,25.823920,26.923920,whisker_trial,6,1,3.2,3,25.823920,0,0,0,0,NaN,0,NaN,25.823920,1,25.923920,26.923920,0,NaN,25.823920,25.823920,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,28.856292,29.956292,whisker_trial,6,1,3.2,3,28.856292,0,0,0,0,NaN,0,NaN,28.856292,1,28.956292,29.956292,0,NaN,28.856292,28.856292,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31.925298,33.025298,whisker_trial,6,1,3.2,3,31.925298,0,0,0,0,NaN,0,NaN,31.925298,1,32.025298,33.025298,0,NaN,31.925298,31.925298,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,34.998745,36.098745,whisker_trial,6,1,3.2,3,34.998745,0,0,0,0,NaN,0,NaN,34.998745,1,35.098745,36.098745,0,NaN,34.998745,34.998745,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,4005.387284,4006.487284,whisker_trial,6,1,3.2,3,4005.387284,0,0,0,0,NaN,0,NaN,4005.387284,1,4005.487284,4006.487284,0,NaN,4005.387284,4005.387284,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,427
428,4008.430656,4009.530656,whisker_trial,6,1,3.2,3,4008.430656,0,0,0,0,NaN,0,NaN,4008.430656,1,4008.530656,4009.530656,0,NaN,4008.430656,4008.430656,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,428
429,4011.488732,4012.588732,whisker_trial,6,1,3.2,3,4011.488732,0,0,0,0,NaN,0,NaN,4011.488732,1,4011.588732,4012.588732,0,NaN,4011.488732,4011.488732,0,passive,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429


### Whisker &  Stimulation
#### Now iterate in each row of short\_units and create a column with tables to append values for number of spikes for each 

In [6]:
### part 1 : with passive trials
#active_trials = trials[trials['context']] == 'passive'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
### part 2 : with active trials
#passive_trials = trials[trials['context']] == 'active'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
# Create a new dataframe 
mouse_name = "AB082_20230630_101353.nwb"

data['pre'] = 0.2
data['post'] = 0.2
data['Mouse name'] = mouse_name



folder = f'Data/{mouse_name}'
os.makedirs(folder, exist_ok=True)

#### Only problem : arrays present in columns are converted to string of array, so careful while preprocessing ####

###### It is better to use parquet files ######
#data.to_csv(f'{mouse_name}_Selectivity_Dataframe.csv', index=True, header=True)
#data.to_pickle(f'{mouse_name}_Selectivity_Dataframe.pkl')
data.to_parquet(f'{folder}/{mouse_name}_Selectivity_Dataframe.parquet', index=False)